# Download of all dependencies for library (restart may be required)

In [ ]:
from google.colab import output

In [ ]:
!pip install git+https://github.com/aimclub/eXplain-NNs &> /dev/null

!wget https://raw.githubusercontent.com/aimclub/eXplain-NNs/main/requirements.txt
!pip install -r requirements.txt
# !pip install torchmetrics &> /dev/null


--2024-01-08 10:19:22--  https://raw.githubusercontent.com/aimclub/eXplain-NNs/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1486 (1.5K) [text/plain]
Saving to: ‘requirements.txt.2’

requirements.txt.2  100%[===================>]   1.45K  --.-KB/s    in 0s      

2024-01-08 10:19:22 (16.4 MB/s) - ‘requirements.txt.2’ saved [1486/1486]



# Dataset load and unzip for further use

We are going to demonstrate the algorithm on the landscape classification from satellite imagery dataset [EuroSAT](https://www.kaggle.com/code/nilesh789/land-cover-classification-with-eurosat-dataset/notebook)

In [ ]:
from google.colab import output

!wget http://madm.dfki.de/files/sentinel/EuroSAT.zip
!unzip EuroSAT.zip

output.clear()

# Usage demonstration

For demonstration of usage of module eXNN.bayes we train two simple networks - one will be more vulnerable to adversarial noise (undertrained), and one will be more robust. In order to distinguish them, we will use DropoutBayesianWrapper from eXNN.bayes in order to evaluate uncertainty of predictions for train and test input.

First, load dataset and required libraries

In [ ]:
from torchvision.datasets import MNIST, ImageFolder
import torch
import torch.nn as nn
import torchvision.transforms as TF
from tqdm.auto import tqdm
import os
import numpy as np
from random import shuffle
from torchvision.models import resnet18

import warnings
warnings.filterwarnings('ignore')


path_to_data = '2750'

dataset = ImageFolder(
    path_to_data,
    transform = TF.Compose([
        TF.Resize((64, 64)),
        TF.ToTensor()
    ])
)

n_obj = 3000
cls_indicies = [(i, obj) for i, obj in enumerate(dataset.targets)]
shuffle(cls_indicies)

cls_indicies = cls_indicies[0:n_obj]

num_classes = len(dataset.classes)

classes_names = dataset.classes

train_ds = torch.utils.data.Subset(dataset, indices = [i[0] for i in cls_indicies[0:int(len(cls_indicies)*0.6)]])
test_ds = torch.utils.data.Subset(dataset, indices = [i[0] for i in cls_indicies[int(len(cls_indicies)*0.6):]])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=36, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=36, shuffle=False)

Define training functions to train two models

In [ ]:
import time

def train_model(model, device = 'cuda', good_case = True):
  if good_case:
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, weight_decay = 0.0001)#, momentum=0.9)
  else:
    optimizer = torch.optim.SGD(model.parameters(), lr = 1e-1, weight_decay = 0, momentum = 0.9)
  criterion = nn.CrossEntropyLoss()
  images, labels = next(iter(train_dl))
  #images = images.view(images.shape[0], -1)
  logps = model(images)
  loss = criterion(logps, labels)
  # train
  if good_case:
    n_epochs = 20
  else:
    n_epochs = 5
  for e in range(n_epochs):
    time_mark = time.time()
    running_loss = 0
    for images, labels in train_dl:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {} - Time taken: {} sec".format(e, running_loss/len(train_dl), round(time.time() - time_mark, 2)))

  return model

Well train model

In [ ]:
model_good = resnet18(num_classes=num_classes)

model_good = train_model(model_good, device = 'cpu', good_case = True)

Epoch 0 - Training loss: 1.9492802000045777 - Time taken: 36.93 sec
Epoch 1 - Training loss: 1.5442372393608093 - Time taken: 34.69 sec
Epoch 2 - Training loss: 1.3513949728012085 - Time taken: 40.52 sec
Epoch 3 - Training loss: 1.2081313419342041 - Time taken: 35.22 sec
Epoch 4 - Training loss: 1.0933070242404939 - Time taken: 35.03 sec
Epoch 5 - Training loss: 0.9831119573116303 - Time taken: 35.17 sec
Epoch 6 - Training loss: 0.8877223074436188 - Time taken: 35.31 sec
Epoch 7 - Training loss: 0.8023216778039932 - Time taken: 34.3 sec
Epoch 8 - Training loss: 0.7429862898588181 - Time taken: 36.59 sec
Epoch 9 - Training loss: 0.6704698818922042 - Time taken: 35.02 sec
Epoch 10 - Training loss: 0.6206120127439498 - Time taken: 38.32 sec
Epoch 11 - Training loss: 0.5655347180366516 - Time taken: 41.13 sec
Epoch 12 - Training loss: 0.4657299715280533 - Time taken: 37.05 sec
Epoch 13 - Training loss: 0.41884350806474685 - Time taken: 36.12 sec
Epoch 14 - Training loss: 0.3951619663834572

Undertrain model

In [ ]:
model_bad = resnet18(num_classes = num_classes)
model_bad = train_model(model_bad, device = 'cpu', good_case = False)

Epoch 0 - Training loss: 3.005459487438202 - Time taken: 32.29 sec
Epoch 1 - Training loss: 1.8501051425933839 - Time taken: 31.88 sec
Epoch 2 - Training loss: 1.5950738978385925 - Time taken: 32.72 sec
Epoch 3 - Training loss: 1.4924990403652192 - Time taken: 32.19 sec
Epoch 4 - Training loss: 1.4497403573989869 - Time taken: 33.98 sec


Here we define one of the most popular adversarial noises - fgsm attack - in order to test the ability to distinguish between "clean" and "noised" examples (as seen by this model) - those for which model give reliable information and those for which it don't.

In [ ]:
# define adversarial attack

def fgsm_attack(model, loss, images, labels, eps, device):

    images = images
    labels = labels
    images.requires_grad = True

    outputs = model.forward(images)

    model.zero_grad()
    cost = loss(outputs, labels).to(device)
    cost.backward()

    attack_images = images + eps*images.grad.sign()
    attack_images = torch.clamp(attack_images, 0, 1)

    return attack_images

def _d(t: torch.Tensor):
    return t.detach().cpu()

In [ ]:
n_batches = 5

train_iterator = iter(train_dl)
batch = [(x[0], x[1]) for _, x in zip(range(n_batches), train_dl)]

train_batch = [torch.cat([i for i,j in batch], dim = 0), torch.cat([j for i,j in batch], dim = 0)]

corrupted_test = fgsm_attack(model = model_good, loss = torch.nn.NLLLoss(), images = train_batch[0], labels = train_batch[1], eps = 1e-2, device = 'cpu')

Let's import DropoutBayesianWrapper in order to give our model the ability to evaluate prediction uncertainty.

In [ ]:
from eXNN.bayes import DropoutBayesianWrapper

model_good_bayes = DropoutBayesianWrapper(model_good, 'beta', p = None, a = 0.6, b = 12.0)

Make predictions for "clean" and "noisy" examples and compare prediction uncertainties for them

In [ ]:
svd_bayes_std_norm = model_good_bayes.predict(train_batch[0], n_iter = 10)['std'].detach().numpy()
svd_bayes_std_corr = model_good_bayes.predict(corrupted_test, n_iter = 10)['std'].detach().numpy()

TypeError: dropout(): argument 'input' (position 1) must be Tensor, not NoneType

In [ ]:
import pandas as pd

res_uncert = pd.DataFrame()
res_uncert['Group'] = ["Normal"]*len(svd_bayes_std_norm) + ["Corrupted"]*len(svd_bayes_std_corr)
res_uncert["Uncert"] = np.concatenate([svd_bayes_std_norm, svd_bayes_std_corr], axis = 0).mean(axis = 1)

res_uncert

Plot prediction uncertainties in order to see the difference between them

In [ ]:
import matplotlib.pyplot as plt

res_uncert.boxplot(column = "Uncert", by = 'Group', figsize = (10, 5))
plt.title("Uncertainty for corrupted and normal data")

In [ ]:
fig, ax = plt.subplots(1, 2)

ind = 0

ax[0].imshow(np.moveaxis(train_batch[0][ind].detach().numpy(), 0, -1))
std1 = str(round(svd_bayes_std_norm[ind].mean(), 2))
ax[0].set_title(f"Prediction std: {std1}")

ax[1].imshow(np.moveaxis(corrupted_test[ind].detach().numpy(), 0, -1))
std1 = str(round(svd_bayes_std_corr[ind].mean(), 2))
ax[1].set_title(f"Prediction std: {std1}")